In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bulletin.commom import static, normalize

pd.set_option('display.max_columns', None)

In [2]:
geral = pd.read_csv('INFORME_EPIDEMIOLOGICO_27_11_Geral.csv', sep=';', converters={'IDADE_ORIGINAL': lambda x: normalize.normalize_number(x,fill=0)}, parse_dates=['DATA_DIAGNOSTICO','DATA_OBITO'], date_parser=pd.to_datetime)
geral.columns = [ normalize.normalize_labels(x) for x in geral.columns ]
geral.shape[0]

272984

In [3]:
obitos = geral.loc[ geral['data_obito'].notnull() ].copy()
obitos.loc[ obitos['data_obito'] < static.se.loc[static.se['se'] == 42, 'inicio'].to_list()[0], 'se' ] = '11 até 41'
obitos.loc[ obitos['data_obito'] > static.se.loc[static.se['se'] == 42, 'fim'].to_list()[0], 'se' ] = '43 até 48'

In [4]:
geral.loc[ geral['data_diagnostico'] < static.se.loc[static.se['se'] == 42, 'inicio'].to_list()[0], 'se' ] = '11 até 41'
geral.loc[ geral['data_diagnostico'] > static.se.loc[static.se['se'] == 42, 'fim'].to_list()[0], 'se' ] = '43 até 48'

In [5]:
faixa_etaria = [10,20,30,40,50,60,70,80,90,100]
faixa_etaria_labels = ['0 a 9','10 a 19','20 a 29','30 a 39','40 a 49','50 a 59','60 a 69','70 a 79','80 a 89','90 a 99','100 ou +']
faixa_etaria_index = dict(enumerate(faixa_etaria_labels))
index_casos = np.digitize(geral['idade_original'], faixa_etaria, right=True)
index_obitos = np.digitize(obitos['idade_original'], faixa_etaria, right=True)

In [6]:
geral_se_casos = geral.groupby(by=['se',index_casos])[['casos']].count().rename(index=faixa_etaria_index)
geral_se_casos.index.names = [ 'se', 'faixa_etaria' ]

In [7]:
geral_se_obitos = obitos.groupby(by=['se',index_obitos])[['obitos']].count().rename(index=faixa_etaria_index)
geral_se_obitos.index.names = [ 'se', 'faixa_etaria' ]

In [8]:
faixa_etaria = geral_se_casos.join(geral_se_obitos).fillna(0)

In [9]:
faixa_etaria['obitos'] = faixa_etaria['obitos'].apply(int)

In [10]:
faixa_etaria.to_excel('faixa_etaria.xlsx')